In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab6.ipynb")

---

<h1><center>SDSE Lab 6 <br><br> Ensemble methods and hyperparameter tuning </center></h1>

---




In this lab exercise you will apply several machine learning classification models to a problem in astrophysics. The problem is described [here](https://satyam5120.medium.com/predicting-a-pulsar-star-using-different-machine-learning-algorithms-d22ee8fc71b4) and [here](https://www.kaggle.com/datasets/colearninglounge/predicting-pulsar-starintermediate). It consists in labeling observations of space objects as either "pulsars" or "not pulsars", based on the properties of a so-called integrated profile and the DM-SNR curve. The definitions of an "integrated profile" and of the "DM-SNR curve" are not important for this activity.

The dataset has 8 feature columns:
1. Mean of the integrated profile.
2. Standard deviation of the integrated profile.
3. Excess kurtosis of the integrated profile.
4. Skewness of the integrated profile.
5. Mean of the DM-SNR curve.
6. Standard deviation of the DM-SNR curve.
7. Excess kurtosis of the DM-SNR curve.
8. Skewness of the DM-SNR curve.

Our goal is to choose a classification model from the ones covered in this class. The procedure will follow these steps:    
1. Load the data.
2. Remove null values.
3. Compute the number of samples per class.
4. Extract a test dataset.
5. Build models:
    + Logistic regression
    + Random forest
    + AdaBoost
    + Gradient Boosted Trees
6. Select and evaluate a final model.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lab6_utils import unpack_gridsearch, plot_grid_result
from hashutils import *
rng_seed = 2434

# 1. Load the data

+ Load the data file `pulsar_data.csv` into a Pandas dataframe called `raw_data`.
+ Save the column headers corresponding to feature names to the variable `features`. Thhese are all except the `target_class`, which is the output column.

**Hints**:

+ [`pd.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)
+ [`DataFrame.columns`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.columns.html)
+ [`Index.drop`](https://pandas.pydata.org/docs/reference/api/pandas.Index.drop.html)

In [ ]:
raw_data = ...
features = ...

In [ ]:
grader.check("q1")

# 2. Remove null values

+ Remove any feature columns of `raw_data` with more than zero null values. Store the result in a new DataFrame called `clean_data`. Do not modify `raw_data`.
+ Store the remaining feature names in a list (or other iterable type) called `clean_features`.

**Hints**: 

+ [`DataFrame.copy`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html)
+ [`np.any`](https://numpy.org/doc/stable/reference/generated/numpy.any.html)
+ [`pd.isnull`](https://pandas.pydata.org/docs/reference/api/pandas.isnull.html)  
+ [`DataFrame.drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) 
+ This part can be done with or without a for loop, depending on your preference.  

In [ ]:
clean_data = ...
clean_features = ...

In [ ]:
grader.check("q2")

# 3. Number of samples per class
Find the number of samples in each of the two classes. Use these variable names:
+ `N0`... number of samples of class 0 (not a pulsar), 
+ `N1`... number of samples of class 1 (pulsar). 

What is the minimum accuracy expected of any non-trivial model? Save your answer to the variable `baseline_acc`.

**Note**: The baseline accuracy should be expressed as a number between 0 and 1, not as a percentage. 

In [ ]:
N0 = ...
N1 = ...
baseline_acc = ...

In [ ]:
grader.check("q3")

# 4. Extract the test dataset

Use Scikit-learn's `train_test_split` method to split `cleandata` into trainging and testing parts. Keep 90% for training and 10% for testing. 

**Notes**: 
+ [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
+ Pass `random_state=rng_seed` to `train_test_split` so that the result is repeatable.
+ `train_test_split` takes as inputs two 2D input and output arrays. You should extract these from `cleandata` prior to calling `train_test_split`.

In [ ]:
from sklearn.model_selection import train_test_split
...
Xtrain, Xtest, ytrain, ytest = ...

In [ ]:
grader.check("q4")

# 5. Logistic regression 

This is a binary classification problem, so it makes sense to begin with logistic regression. 

## 5.1. Training

In a previous lab we found that logistic regression can benefit from a normalization (or scaling) of the inputs. 
Create a pipeline that includes a `StandardScaler` (name it `'scaler'`), followed by `LogisticRegression` model (name it `'model'`). 

The logistic regression model should be created with these parameters:
+ `solver='liblinear'`
+ `random_state=rng_seed`

Train the model. 

**Note**: 
+ Import Scikit-learn's `Pipeline`, `LogisticRegression`, and `StandardScaler` classes.
+ Provide the appropriate imports.

In [ ]:
...
logreg_pipe = Pipeline([
    ('scaler' , ... ) ,
    ('model' , ... )
])
...

In [ ]:
grader.check("q5p1")

## 5.2. Hyperparameter tuning with grid search

The "hyperparameter tuning problem" is to select the values of the hyperparameters that maximize the performance of the model. To solve an optimization problem by "grid search" is to exhaustively search over a grid of points covering the feasible space. In this section we will solve the hyperparameter tuning problem with grid search. Each setting of the hyperparameters will be evaluated using K-fold cross-validation (as opposed to a validation dataset).

Scikit-learn provides an implementation of grid search with K-fold cross-validation in the [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class. See the documentation of this class for a detailed explanation. 

Our model will be the logistic regression pipeline from part 5.1. The hyperparameters and values to search over are the following. 

+ `penalty`: the regularization function. Try `'l1'` (lasso) and `'l2'` (ridge).
+ `C`: the regularization weight. This is equivalent to $1/\lambda$ in our notation. Try five logarithmically spaced values between 0.001 and 10 (i.e. $10^{-3}$, $10^{-2}$, $10^{-1}$, $1$, and $10$). 

Running grid search with Scikit-learn involves two steps: creating the `GridSearchCV` object, and then executing the algorithm with the `fit` method. Create a `GridSearchCV` with these arguments:
+ `param_grid=param_grid`. See explanation below.
+ `scoring='accuracy'`. Use accuracy as the performance metric.
+ `cv=3`. Three-fold cross-validation.
+ `refit='accuracy'`. Directs the function to train a final "best model" with the optimal hyperparameter setting and all of the training data, and to evaluate that model's accuracy.

The list of candidate values for each of the hyperparameters is passed to `GridSearchCV` as a dictionary (`param_grid`) with entries parameter name:list of parameter values. When working with a pipeline, the parameter name must be prefixed with the name of the model ('model' in our case), and two underscores (`__`). Thus the `param_grid` argument to  `GridSearchCV` should look like this:

```python
param_grid = {
    'model__penalty' : ...,
    'model__C' : ... }
```

The second step is to run the grid search algorithm passing the training data to the `fit` method on the `GridSearchCV` object. 

Implement grid search in the cell below.

**Note**: 

+ Import Scikit-learn's `GridSearchCV` class

In [ ]:
...
param_grid = ...
gs_logreg = GridSearchCV(...)
gs_logreg.fit(...)


`lab6_utils.py` contains useful functions for unpacking and plotting the result of the grid search. 

Below we use the `unpack_gridsearch` method to extract information from the grid search solution. This method returns a dictionary with the following entries:
+ `scoregrid`: The values of the accuracies over the hyperparameter grid. 
+ `best_params`: A dictionary with the best-case hyperparameter values.
+ `best_estimator`: The logistic regression pipeline corresponding to the best hyperparameters. 
+ `best_score`: The accuracy of the best model.
+ `param_grid`: The parameter grid that was used in the grid search.

`plot_grid_result` plots the results of the grid search.


<!-- X, best_params, best_estimator, best_score -->

In [ ]:
logreg_result = unpack_gridsearch(gs_logreg)
plot_grid_result(logreg_result)

In [ ]:
grader.check("q5p2")

## 5.3. Package it into a function

Next we will repeat this process with several other classification models. To keep the code clean, we will first collect the steps into a single function, which we'll call `hypersolve`. This function receives as inputs:
1. The classifier object, e.g. `LogisticRegression(solver='liblinear',random_state=rng_seed)`
2. The `param_grid` dictionary that defines the search space for `GridSearchCV`.

It should:
1. Create a pipeline containing a standard scaler and the model.
2. Construct the `GridSearchCV` object as was done in part 5.2
3. Run the grid se

Run `fit` on the grid search object, using the training data
4. Run `unpack_gridsearch` the dictionary with results.
5. Plot the result with `plot_grid_result`
6. Return the `result` dictionary.


**Note**: This part has no autograder test. Failures in subsequent parts may be due to errors in `hypersolve`.

In [ ]:

def hypersolve(model,param_grid):
    # 1. Create the pipeline model
    pipe = Pipeline([
    ...
    ])

    # 2. Construct the `GridSearchCV` object as was done in part 5.2
    gs = GridSearchCV(...)
    
    # 3. Run the grid search
    gs = gs.fit(...)
    
    # 4. Run `unpack_gridsearch` to obtain the results dictionary.
    result = unpack_gridsearch(...)

    # 5. Plot the result with `plot_grid_result`
    plot_grid_result(...)

    # 6. return result
    return ...

# 6. Random forest

Use grid search to optimize a random forests model over the following parameter ranges:
+ `max_features`: This is the number of features to search over when splitting a node. Test 3 features and all features. 
+ `n_estimators`: Test 2, 42, and 82 trees. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_grid_rf = ...

model = RandomForestClassifier(random_state=rng_seed)

result_rf = hypersolve(...)

In [ ]:
grader.check("q6")

# 7. AdaBoost

Repeat the grid search, this time using an AdaBoost model. Use the following values for the parameter grid:
+ `learning_rate: [0.01,0.1]`. $\lambda$ in the notation of the lecture.
+ `n_estimators: [20,60,100]`. $M$ in the notation of the lecture, unless perfect prediction is reached earlier. 

**Note**: 

+ Remember to set the random state for the model in the model's constructor.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

param_grid_ab = ...

model = AdaBoostClassifier(random_state=rng_seed)

result_ab = hypersolve(...)

In [ ]:
grader.check("q7")

# 8. Gradient Boosted Trees

Use the following parameters for the hyper-parameter search. These have the same role as in AdaBoost.
+ `learning_rate: [0.01,0.1]`. 
+ `n_estimators: [20,60,100]`. 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid_gbt = ...

model = ...

result_gbt = hypersolve(...)

In [ ]:
grader.check("q8")

# 9. Final model selection

We have now built four separate classifiers: logistic regression, random forest, AdaBoost, and Gradient boosted trees. These are stored below in the `all_models` list. Select from this ist the classifier with the best score. Save the corresponding name, model, and score to variables `best_name`, `best_model`, and `best_score` respectively. Then compute the test accuracy for that model, using its `predict` and `accuracy_score` functions. Store the result as `test_accuracy`.

In [ ]:
from sklearn.metrics import accuracy_score

# re-solve logistic regression
param_grid_lr = {
    'model__penalty' : ['l1','l2'],
    'model__C' : np.logspace(-3, 1, 5) }
result_lr = hypersolve( LogisticRegression(solver='liblinear',random_state=rng_seed), param_grid_lr)

all_models = [
    ('logreg',result_lr['best_estimator'], result_lr['best_score']),
    ('rf',result_rf['best_estimator'], result_rf['best_score']),
    ('ab',result_ab['best_estimator'], result_ab['best_score']),
    ('gbt',result_gbt['best_estimator'], result_gbt['best_score']),
]

In [ ]:
...
best_name = ...
best_model = ...
best_score = ...

ypred = ...
test_accuracy = ...
test_accuracy

In [ ]:
grader.check("q9")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)